In [1]:
import pandas as pd
import pandas

In [2]:

# df['num_redundant_layers']

In [2]:
%matplotlib inline
from numpy import array, linspace ; from numpy.random import randint
from matplotlib.pyplot import hist, xticks, show
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
# synthesize some data 
font = {
    'family' : 'serif',
    'weight':'normal',
        'size'   : 12}

matplotlib.rc('font', **font)
import seaborn as sns
pallete = sns.color_palette("Set1", 4)
dgl_color = pallete[0]
quiver_color = pallete[1]
split_color = pallete[2]
p3_color = pallete[3]
hatches = list('-*/o')

pallete = sns.color_palette("Set1", 5)
pallete

[(0.8941176470588236, 0.10196078431372549, 0.10980392156862745),
 (0.21568627450980393, 0.49411764705882355, 0.7215686274509804),
 (0.30196078431372547, 0.6862745098039216, 0.2901960784313726),
 (0.596078431372549, 0.3058823529411765, 0.6392156862745098),
 (1.0, 0.4980392156862745, 0.0)]

In [4]:
import pandas as pd
machine_name = 'p3.8xlarge/nvlink'
# machine_name = 'g4dn.12xlarge/pcie'
df = pd.read_csv(f'march_logs/{machine_name}_default.csv')
# print(df)
d = {'system': 1, 'b': 2, 'c': 3}
dfs = []
idx = ['model','system']
dgl_cache_pers = [1, 0, 0, 0,]
graphs = ['ogbn-papers100M']
graphs = ['ogbn-papers100M','com-orkut','com-friendster']
graphs = ['papers100M','orkut','friendster']
graphs = ['orkut', 'papers100M', 'friendster']
# graphs = []
# , 'com-orkut', 'com-friendster']
for graph_name in graphs:
    idx.append(f'sampling_{graph_name}')
    idx.append(f'loading_{graph_name}')
    idx.append(f'training_{graph_name}')
    idx.append(f'total_{graph_name}')
batch_size = 256
system_label = {}
system_label = {'dgl':'DGL','p3':'Push-Pull', 'quiver':'Quiver',\
                    'dist_cache':'dist_cache','split':'\\name'}
graph_label = {'ogbn-products':"Products", "ogbn-papers100M":"Papers100M", \
                   "com-orkut":"Orkut", "com-friendster":"Friendster"}
graph_label = {'ogbn-products':"Products", "papers100M":"Papers100M", \
                   "orkut":"Orkut", "friendster":"Friendster"}
dgl_time = {}
for dataset_id,graph_name in enumerate(graphs):
    split_time = {} 
    for i, system in enumerate(['split','dgl','p3','quiver','dist_cache','split']):
        out = {}
        out['graph_name'] = graph_label[graph_name]
        out['system'] = system_label[system]
        for model in ["sage", "gat"]:
            dgl_cache_per = dgl_cache_pers[dataset_id]
            dataset_df = df[(df['graph_name'] == graph_name) & (df['model'] == model)\
                        ]
            
            if system == "dgl":
                row = dataset_df[dataset_df['system'].str.contains('dgl')]
#                 print(row)
            if system == "dist_cache":
                row = dataset_df[dataset_df['system'].str.contains('dist_cache')]
            if system == "quiver":
            
                row = dataset_df[dataset_df['system'].str.contains('quiver')]
            if system == "split":
                groot_row = dataset_df[dataset_df['system'].str.contains('split') \
                                       & (dataset_df['partition_type'] == 'ndst_efreq_xbal')]
                if len(groot_row) != 1:
                    row = groot_row[~((groot_row['cache_size'] == '0') | (groot_row['cache_size'] == '1'))]
                else:
                    row = groot_row
                if(len(row) != 1):
                    row = row[row['num_redundant_layers'] == 0]
            if system == "p3":
                row = dataset_df[dataset_df['system'].str.contains('p3')]
            if len(row) != 1:
                print(row, graph_name, system)
            sampling_time = row['sampling (s)'].item()
            if sampling_time != "oom":
                n_epochs = float(row['num_epoch'].item())
                sampling_time = float(sampling_time)
                data_loading = float(row['feature (s)'].item())
                training_time = (float(row['forward (s)'].item()) + float(row['backward (s)'].item()))
                out[f'sampling_{model}'] = "{:.2f}".format(sampling_time)
                out[f'loading_{model}'] = "{:.2f}".format(data_loading)
                out[f'data_{model}'] = "{:.2f}".format(training_time)
                training_time = sampling_time + data_loading + training_time
                out[f'total_{model}'] = "{:.2f}".format(training_time)
                if system == "split":
                    split_time[model] = training_time
                    out[f'speed_up_{model}']  = ''
                else:
                    out[f'speed_up_{model}'] = 'x{:.1f}'.format(training_time/split_time[model])
                    
            else:
                out[f'sampling_{model}'] = "OOM"
                out[f'loading_{model}'] = "OOM"
                out[f'data_{model}'] = "OOM"
                out[f'total_{model}'] = "OOM"
                  
        if i != 0:
            dfs.append(pd.Series(out).to_frame().T)
final = (pd.concat(dfs, ignore_index = True))
print(final.to_csv(index = False))
# print(final)
print(machine_name)

graph_name,system,sampling_sage,loading_sage,data_sage,total_sage,speed_up_sage,sampling_gat,loading_gat,data_gat,total_gat,speed_up_gat
Orkut,DGL,4.06,60.09,9.23,73.38,x3.9,4.06,60.02,17.15,81.23,x3.2
Orkut,Push-Pull,4.04,1.55,8.53,14.12,x0.7,4.04,1.65,37.62,43.31,x1.7
Orkut,Quiver,6.06,4.99,8.67,19.72,x1.0,6.24,63.69,16.41,86.34,x3.4
Orkut,dist_cache,6.10,1.36,8.63,16.09,x0.8,6.24,6.16,16.28,28.68,x1.1
Orkut,\name,4.32,0.09,14.62,19.03,,4.31,0.09,20.70,25.10,
Papers100M,DGL,3.15,8.79,10.33,22.27,x1.2,2.94,8.56,30.82,42.32,x1.1
Papers100M,Push-Pull,3.28,11.51,25.84,40.63,x2.1,3.33,11.30,65.72,80.35,x2.1
Papers100M,Quiver,10.59,15.98,11.14,37.71,x2.0,10.04,14.96,30.49,55.49,x1.4
Papers100M,dist_cache,10.68,7.28,11.16,29.12,x1.5,10.07,7.64,30.38,48.09,x1.3
Papers100M,\name,4.45,2.86,11.90,19.21,,4.66,2.40,31.23,38.29,
Friendster,DGL,62.71,283.40,61.11,407.22,x2.3,62.56,284.76,245.94,593.26,x1.6
Friendster,Push-Pull,85.90,350.78,151.47,588.15,x3.3,76.48,351.35,613.78,1041.61,x2.9
Friends